<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=f69f535533988d46e8eab0dff5fac08dfd642577eee15eb100e62fd053df37da
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-07 09:50:19
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.05 L
Current:  1.42 C
-------------------
Today PnL: -1.21 L (-0.85%)
Current PnL: -22.85 L (-14.98%)
CY Booked + Current PnL: -10.20 L (-6.69%)
-------------------
Total profit:  1.70 L
Total loss:  -24.56 L
-------------------
Total Booked + Current PnL: 16.78 L (13.36%)
Total Booked PnL: 39.63 L (31.55%)
Curr Year Booked PnL: 12.65 L (8.93%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 89.84 L (63.39%)
Deployed:  1.26 C
Current:  1.42 C
CAGR/XIRR %: 7.04%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RELIANCE,1533.00,-6.53,68.0,X-LC,6.49,233532.0,23166.0,5605.0,0.06,11.01,2.40,13.68,37.0,4.13,1.66,29.36,XY25,NTT,REFINERIES
78,TTKPRESTIG,770.00,100.85,50.0,M-SC,1.92,86578.0,-14199.0,14294.0,-0.37,-14.09,16.51,0.09,245.0,-0.99,0.62,13.74,OX40N,NTT,DURABLES
79,UNIONBANK,163.00,-6.98,61.0,M-LC,11.66,169208.0,28368.0,16126.0,-0.89,20.14,9.53,31.59,66.0,1.76,1.20,51.97,XY24,NTT,BANKS
2,ABBOTINDIA,35195.00,-10.65,41.0,X-MC,2.81,88200.0,-2478.0,17384.0,-0.19,-2.73,19.71,16.44,101.0,-0.14,0.63,16.89,X40,ATH,PHARMA
33,ICICIGI,2252.93,-14.54,53.0,X-MC,5.78,146890.0,10757.0,19830.0,-0.80,7.90,13.50,22.47,91.0,0.54,1.04,21.91,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VALIANTORG,838.00,-285.90,43.0,H-SC,11.26,103326.0,-60279.0,181595.0,1.96,-36.84,175.75,74.15,139.0,-0.33,0.73,32.45,XR,NTT,CHEMICALS
84,VOLTAS,1530.00,-6.53,33.0,X-MC,4.35,196125.0,4383.0,33380.0,1.17,2.29,17.02,19.69,99.0,0.13,1.39,9.24,XY25,NTT,AC
37,INDIGOPNTS,1408.00,89.17,35.0,M-SC,6.39,123994.0,-50565.0,50602.0,0.92,-28.97,40.81,0.02,221.0,-1.00,0.88,7.85,OX40N,NTT,PAINTS
24,EASEMYTRIP,26.40,-18.06,50.0,M-SC,27.35,78845.0,-97970.0,178765.0,0.75,-55.41,226.73,45.70,197.0,-0.55,0.56,2.93,XY24,NTT,TRAVEL
75,TCS,4389.97,-29.01,49.0,X-LC,9.75,283025.0,-62931.0,129625.0,0.69,-18.19,45.80,19.28,1.0,-0.49,2.01,4.63,X40,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,SURYODAY,240.0,55.70,42.0,H-SC,15.72,143392.0,-35679.0,101407.0,-5.74,-19.92,70.72,36.71,135.0,-0.35,1.02,42.27,XR,NTT,BANKS
71,SYMPHONY,1306.0,-45.32,41.0,M-SC,16.76,116721.0,-54420.0,54369.0,-5.09,-31.80,46.58,-0.03,196.0,-1.00,0.83,1.76,OX40N,NTT,DURABLES
1,ABB,7934.0,-42.10,37.0,H-LC,5.06,242064.0,-19555.0,138775.0,-3.54,-7.47,57.33,45.57,7.0,-0.14,1.72,3.00,AR,NTT,ELECTRICAL
67,SIS,528.0,1980.31,40.0,H-SC,2.56,83946.0,-27086.0,50695.0,-3.01,-24.39,60.39,21.26,156.0,-0.53,0.60,13.42,OX40N,NTT,MISC
17,CERA,9475.0,-24.34,42.0,H-SC,1.84,137988.0,-37915.0,79936.0,-2.99,-21.55,57.93,23.89,149.0,-0.47,0.98,19.17,OX40N,NTT,CERAMICS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,KPIGREEN,731.05,5.95,52.0,H-SC,7.95,126252.0,955.0,57975.0,-1.91,0.76,45.92,47.03,141.0,0.02,0.9,58.08,MH,ATH,POWER


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,100.85,50.0,M-SC,1.92,86578.0,-14199.0,14294.0,-0.37,-14.09,16.51,0.09,245.0,-0.99,0.62,13.74,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,42.0,H-SC,2.72,220401.0,-49266.0,85604.0,-1.33,-18.27,38.84,13.47,138.0,-0.58,1.57,12.07,XY24,NTT,PAINTS
17,CERA,9475.0,-24.34,42.0,H-SC,1.84,137988.0,-37915.0,79936.0,-2.99,-21.55,57.93,23.89,149.0,-0.47,0.98,19.17,OX40N,NTT,CERAMICS
42,JCHAC,2282.0,19921.74,62.0,M-SC,1.50,97838.0,-29967.0,29958.0,-0.05,-23.45,30.62,-0.01,232.0,-1.00,0.70,10.79,OX40N,NTT,AC
67,SIS,528.0,1980.31,40.0,H-SC,2.56,83946.0,-27086.0,50695.0,-3.01,-24.39,60.39,21.26,156.0,-0.53,0.60,13.42,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UNIONBANK,163.0,-6.98,61.0,M-LC,11.66,169208.0,28368.0,16126.0,-0.89,20.14,9.53,31.59,66.0,1.76,1.2,51.97,XY24,NTT,BANKS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,49.16,56.0,M-MC,10.23,234881.0,9919.0,157746.0,-1.09,4.41,67.16,74.53,192.0,0.06,1.67,38.08,XY24,BTT,STEEL
46,KPIGREEN,731.05,5.95,52.0,H-SC,7.95,126252.0,955.0,57975.0,-1.91,0.76,45.92,47.03,141.0,0.02,0.90,58.08,MH,ATH,POWER
79,UNIONBANK,163.00,-6.98,61.0,M-LC,11.66,169208.0,28368.0,16126.0,-0.89,20.14,9.53,31.59,66.0,1.76,1.20,51.97,XY24,NTT,BANKS
85,WHIRLPOOL,2270.00,-41.17,50.0,M-SC,6.18,122020.0,7222.0,86817.0,-1.77,6.29,71.15,81.92,223.0,0.08,0.87,44.45,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.71,51.0,H-SC,5.40,129325.0,5989.0,125639.0,-1.33,4.86,97.15,106.72,119.0,0.05,0.92,29.37,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WHIRLPOOL,2270.00,-41.17,50.0,M-SC,6.18,122020.0,7222.0,86817.0,-1.77,6.29,71.15,81.92,223.0,0.08,0.87,44.45,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.71,51.0,H-SC,5.40,129325.0,5989.0,125639.0,-1.33,4.86,97.15,106.72,119.0,0.05,0.92,29.37,AR,ATH,MISC
46,KPIGREEN,731.05,5.95,52.0,H-SC,7.95,126252.0,955.0,57975.0,-1.91,0.76,45.92,47.03,141.0,0.02,0.90,58.08,MH,ATH,POWER
86,WIPRO,420.00,-16.52,39.0,M-LC,5.41,147089.0,-3856.0,113317.0,-1.17,-2.55,77.04,72.51,53.0,-0.03,1.05,3.31,XR,NTT,IT
31,HINDZINC,730.22,24.04,41.0,M-LC,9.69,202036.0,-3040.0,115605.0,-0.59,-1.48,57.22,54.89,52.0,-0.03,1.44,22.26,X5K,ATH,METALS


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-53.17,22.0,X-LC,32.34,148050.0,-128473.0,238553.0,0.33,-46.46,161.13,39.81,54.0,-0.54,1.05,4.35,XY24,NTT,AUTO
12,BATAINDIA,2096.00,-41.87,24.0,X-SC,7.80,86826.0,-41844.0,87139.0,-0.44,-32.52,100.36,35.20,219.0,-0.48,0.62,0.00,X40,NTT,FOOTWEAR
23,DMART,5391.45,-20.90,26.0,X-LC,7.55,201515.0,-8892.0,68052.0,-1.27,-4.23,33.77,28.12,38.0,-0.13,1.43,18.40,X40N,ATH,FMCG
44,JSWINFRA,342.00,-24.16,27.0,X-MC,4.55,190485.0,-9862.0,40364.0,-0.60,-4.92,21.19,15.23,178.0,-0.24,1.35,25.74,X40N,NTT,REALTY
52,PGHH,17907.65,-32.40,30.0,X-MC,0.14,195645.0,-5175.0,72976.0,-0.90,-2.58,37.30,33.76,80.0,-0.07,1.39,2.49,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,PGHH,17907.65,-32.40,30.0,X-MC,0.14,195645.0,-5175.0,72976.0,-0.90,-2.58,37.30,33.76,80.0,-0.07,1.39,2.49,X40,ATH,FMCG
41,ITC,452.00,-37.92,48.0,X-LC,1.05,200025.0,-113.0,21003.0,0.38,-0.06,10.50,10.44,4.0,-0.01,1.42,5.50,X40,NTT,FMCG
66,SIEMENS,4671.50,-5.39,38.0,H-LC,1.39,152115.0,-33980.0,81458.0,-1.34,-18.26,53.55,25.51,15.0,-0.42,1.08,12.10,AR,ATH,ELECTRICAL
28,HAVELLS,2069.16,-9.16,34.0,X-MC,3.23,303366.0,-24625.0,131145.0,0.16,-7.51,43.23,32.48,92.0,-0.19,2.16,3.06,X40,ATH,ELECTRICAL
80,UNITDSPR,1644.00,-6.66,63.0,X-LC,3.97,244842.0,10698.0,37926.0,0.49,4.57,15.49,20.77,86.0,0.28,1.74,11.80,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,-41.87,24.0,X-SC,7.80,86826.0,-41844.0,87139.0,-0.44,-32.52,100.36,35.20,219.0,-0.48,0.62,0.00,X40,NTT,FOOTWEAR
53,QUESS,986.00,-50.96,31.0,X-SC,37.46,48493.0,-16513.0,167441.0,-0.75,-25.40,345.29,232.18,198.0,-0.10,0.34,0.00,XY24,NTT,MISC
51,PAGEIND,51769.93,-30.09,31.0,X-MC,7.43,79000.0,-4510.0,24537.0,-0.65,-5.40,31.06,23.98,82.0,-0.18,0.56,0.10,X40,ATH,APPARELS
20,COLPAL,3726.84,-7.18,40.0,X-MC,7.28,216760.0,-46605.0,155915.0,-0.28,-17.70,71.93,41.51,84.0,-0.30,1.54,1.72,XY25,ATH,FMCG
3,ACC,3906.00,-38.50,42.0,X-MC,2.38,182400.0,-55551.0,208191.0,-0.46,-23.35,114.14,64.15,174.0,-0.27,1.30,2.32,XY24,BTT,CEMENT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-50.96,31.0,X-SC,37.46,48493.0,-16513.0,167441.0,-0.75,-25.40,345.29,232.18,198.0,-0.10,0.34,0.00,XY24,NTT,MISC
51,PAGEIND,51769.93,-30.09,31.0,X-MC,7.43,79000.0,-4510.0,24537.0,-0.65,-5.40,31.06,23.98,82.0,-0.18,0.56,0.10,X40,ATH,APPARELS
57,RELAXO,1176.00,-40.85,40.0,X-SC,4.23,80643.0,-64517.0,140448.0,-1.93,-44.45,174.16,52.31,136.0,-0.46,0.57,7.94,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-41.87,24.0,X-SC,7.80,86826.0,-41844.0,87139.0,-0.44,-32.52,100.36,35.20,219.0,-0.48,0.62,0.00,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-10.65,41.0,X-MC,2.81,88200.0,-2478.0,17384.0,-0.19,-2.73,19.71,16.44,101.0,-0.14,0.63,16.89,X40,ATH,PHARMA


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-29.01,49.0,X-LC,9.75,283025.0,-62931.0,129625.0,0.69,-18.19,45.80,19.28,1.0,-0.49,2.01,4.63,X40,ATH,IT
39,INFY,2275.00,-20.32,45.0,X-LC,5.00,310085.0,-2803.0,174485.0,-0.74,-0.90,56.27,54.87,3.0,-0.02,2.20,7.31,X40,BTT,IT
41,ITC,452.00,-37.92,48.0,X-LC,1.05,200025.0,-113.0,21003.0,0.38,-0.06,10.50,10.44,4.0,-0.01,1.42,5.50,X40,NTT,FMCG
83,VBL,671.64,-16.43,54.0,X-LC,6.31,300817.0,-15025.0,127697.0,-0.07,-4.76,42.45,35.67,5.0,-0.12,2.14,8.44,X40N,ATH,FMCG
1,ABB,7934.00,-42.10,37.0,H-LC,5.06,242064.0,-19555.0,138775.0,-3.54,-7.47,57.33,45.57,7.0,-0.14,1.72,3.00,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-49.70,33.0,L-SC,27.48,73594.0,-39955.0,80004.0,-2.24,-35.19,108.71,35.27,268.0,-0.50,0.52,80.12,XR,NTT,HOTELS
7,ASIANTILES,137.00,7338.89,62.0,L-SC,12.29,81619.0,-12191.0,88810.0,-2.16,-13.00,108.81,81.67,269.0,-0.14,0.58,58.06,XR,NTT,CERAMICS
49,MASFIN,398.61,-20.51,45.0,H-SC,8.31,90540.0,-7440.0,29045.0,-0.79,-7.59,32.08,22.05,152.0,-0.26,0.64,31.61,XR,ATH,FINANCE
85,WHIRLPOOL,2270.00,-41.17,50.0,M-SC,6.18,122020.0,7222.0,86817.0,-1.77,6.29,71.15,81.92,223.0,0.08,0.87,44.45,XR,NTT,DURABLES
46,KPIGREEN,731.05,5.95,52.0,H-SC,7.95,126252.0,955.0,57975.0,-1.91,0.76,45.92,47.03,141.0,0.02,0.90,58.08,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VAIBHAVGBL,521.00,71.90,63.0,H-SC,3.95,149234.0,-33541.0,144608.0,-0.51,-18.35,96.90,60.77,125.0,-0.23,1.06,34.40,XR,NTT,JEWELLERY
46,KPIGREEN,731.05,5.95,52.0,H-SC,7.95,126252.0,955.0,57975.0,-1.91,0.76,45.92,47.03,141.0,0.02,0.90,58.08,MH,ATH,POWER
68,SONACOMS,806.63,-32.59,63.0,M-SC,6.47,83948.0,-17211.0,57211.0,-0.66,-17.01,68.15,39.54,202.0,-0.30,0.60,18.37,AR,ATH,AUTO
7,ASIANTILES,137.00,7338.89,62.0,L-SC,12.29,81619.0,-12191.0,88810.0,-2.16,-13.00,108.81,81.67,269.0,-0.14,0.58,58.06,XR,NTT,CERAMICS
62,SAMMAANCAP,326.00,-195.45,66.0,M-SC,33.92,165573.0,16353.0,127822.0,-0.72,10.96,77.20,96.62,208.0,0.13,1.18,79.19,XY25,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.75
1,25,44.44
2,50,75.50


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.64
LC    33.27
MC    23.09
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.89
X40      16.31
X40N     12.46
XY25     11.99
XR        9.84
AR        7.80
OX40N     6.54
X200      1.79
X5K       1.44
SR        1.04
MH        0.90
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.14
X-LC    22.60
X-MC    18.77
M-SC    12.58
X-SC     5.49
M-LC     5.10
H-LC     4.59
H-MC     2.33
M-MC     1.67
L-SC     1.43
L-LC     0.98
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.37,-4.67,38.50
IT,12.46,-20.03,84.33
FINANCE,12.11,-11.02,61.34
MISC,7.74,-19.24,79.39
ELECTRICAL,5.73,-12.04,51.76
PAINTS,5.63,-16.78,34.19
BANKS,4.00,-20.01,89.83
INSURANCE,3.85,-1.93,39.94
AUTO,2.72,-50.44,113.23


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3369778.0,22
XR,1271852.0,13
AR,1189683.0,9
X40,862175.0,12
X40N,714158.0,9
OX40N,566847.0,9
XY25,487981.0,8
SR,274700.0,2
X5K,115605.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3432848.0,24
M-SC,1278950.0,15
X-MC,1232599.0,14
X-LC,1100044.0,13
X-SC,576004.0,6
H-LC,293128.0,3
M-LC,292383.0,4
H-MC,261197.0,2
L-SC,259416.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1209454.0      6
           AR         844574.0      5
           XR         795249.0      7
M-SC       XY24       740419.0      6
X-MC       XY24       576364.0      4
X-LC       X40        467155.0      5
X-SC       X40N       321424.0      4
X-LC       XY24       317157.0      2
X-MC       X40        307881.0      6
H-SC       SR         274700.0      2
           OX40N      250896.0      3
X-LC       X40N       233675.0      3
H-LC       AR         220233.0      2
X-MC       XY25       189295.0      2
H-MC       XY24       185071.0      1
L-SC       XR         168814.0      2
X-SC       XY24       167441.0      1
X-MC       X40N       159059.0      2
M-MC       XY24       157746.0      1
M-SC       OX40N      149223.0      4
           XR         136610.0      2
           XY25       127822.0      1
           AR         124876.0      2
M-LC       X5K        115605.0      1
           XR         113317.0      1
L-SC       OX40N       90602.0      1
X-SC       X40         87139.0      1
X-LC       XY25        82057.0      3
H-MC       OX40N       76126.0      1
H-LC       X200        72895.0      1
H-SC       MH          57975.0      1
L-MC       XR          57862.0      1
M-LC       XY25        47335.0      1
L-LC       XY25        41472.0      1
M-LC       XY24        16126.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 56.0 seconds
